# Simple Chat with PydanticAI

This notebook demonstrates a basic chat interaction using PydanticAI framework with Ollama models.

PydanticAI is a Python agent framework designed to make it easier to build production-grade applications with LLMs.
It provides type-safe agent interactions, structured outputs, and dependency injection.

## Setup Requirements
- Ollama server running (configured in config.py)
- PydanticAI installed (`pip install pydantic-ai`)
- Model available in Ollama (e.g., 'gpt-oss', 'llama2')

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

# Import shared configuration
from config import config

print("📋 Configuration loaded:")
print(f"   Ollama URL: {config.ollama_base_url}")
print(f"   Model: {config.model_name}")
print(f"   Temperature: {config.temperature}")

In [ ]:
# Test Ollama connection before proceeding
print("🔍 Testing Ollama connection...")

if config.validate_ollama_connection():
    print("✅ Ollama server is accessible")
    
    # Show available models
    models = config.get_available_models()
    if models:
        print(f"📦 Available models: {', '.join(models[:5])}...")  # Show first 5
        if config.model_name in models:
            print(f"✅ Target model '{config.model_name}' is available")
        else:
            print(f"⚠️  Target model '{config.model_name}' not found. Using first available.")
    else:
        print("⚠️  Could not retrieve model list")
else:
    print("❌ Ollama server not accessible")
    print(f"   Check if server is running at: {config.ollama_base_url}")
    print("   You may need to start Ollama or check network connectivity")

In [ ]:
# Import PydanticAI
try:
    from pydantic_ai import Agent
    from pydantic_ai.models import OllamaModel
    print("✅ PydanticAI imported successfully")
except ImportError as e:
    print("❌ PydanticAI not installed or import failed")
    print("   Install with: pip install pydantic-ai")
    print(f"   Error: {e}")
    raise

In [ ]:
# Initialize PydanticAI Agent with Ollama
try:
    # Create Ollama model connection
    model = OllamaModel(
        model_name=config.model_name,
        base_url=config.ollama_base_url.rstrip('/'),  # Remove trailing slash if present
    )
    
    # Create a simple chat agent
    chat_agent = Agent(
        model=model,
        system_prompt="You are a helpful AI assistant. Respond clearly and concisely.",
    )
    
    print("✅ PydanticAI agent initialized successfully!")
    print(f"   Model: {config.model_name}")
    print(f"   Base URL: {config.ollama_base_url}")
    
except Exception as e:
    print(f"❌ Failed to initialize PydanticAI agent: {e}")
    print("\nTroubleshooting:")
    print("1. Ensure Ollama server is running")
    print("2. Check if the model is available in Ollama")
    print("3. Verify network connectivity")

In [ ]:
# Simple chat function
async def simple_chat(message: str):
    """Send a message to the agent and get a response."""
    try:
        result = await chat_agent.run(message)
        return result.data
    except Exception as e:
        return f"Error: {e}"

# Test the chat function
import asyncio

async def test_chat():
    print("🤖 Testing simple chat...")
    
    # Test message
    test_message = "Hello! Can you tell me what you are in one sentence?"
    print(f"👤 User: {test_message}")
    
    response = await simple_chat(test_message)
    print(f"🤖 Assistant: {response}")
    
    return response

# Run the test
try:
    if 'chat_agent' in locals():
        response = await test_chat()
    else:
        print("⚠️  Agent not initialized. Please run the previous cell first.")
except Exception as e:
    print(f"❌ Error during chat test: {e}")

In [ ]:
# Interactive chat loop
async def interactive_chat():
    """Start an interactive chat session."""
    print("💬 Interactive Chat Started!")
    print("Type 'quit', 'exit', or 'bye' to end the conversation")
    print("-" * 50)
    
    while True:
        try:
            # Get user input
            user_input = input("👤 You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', '']:
                print("👋 Goodbye!")
                break
            
            # Get AI response
            print("🤖 Assistant: ", end="", flush=True)
            response = await simple_chat(user_input)
            print(response)
            print()
            
        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("Try again or type 'quit' to exit")

# Instructions for interactive chat
print("📝 To start interactive chat, run:")
print("await interactive_chat()")
print("\nNote: This works best in Jupyter environments that support async/await")

In [ ]:
# Example: Structured response with Pydantic models
from pydantic import BaseModel
from typing import List

class ChatResponse(BaseModel):
    """Structured response model for chat interactions."""
    message: str
    confidence: float
    keywords: List[str]

# Create an agent that returns structured data
structured_agent = Agent(
    model=model,
    result_type=ChatResponse,
    system_prompt="""You are a helpful assistant that provides structured responses.
    For each response, include:
    - message: Your main response
    - confidence: A confidence score between 0.0 and 1.0
    - keywords: Key terms or concepts from the conversation
    """
)

async def structured_chat(message: str):
    """Get a structured response from the agent."""
    try:
        result = await structured_agent.run(message)
        return result.data
    except Exception as e:
        return f"Error: {e}"

# Test structured response
async def test_structured_chat():
    print("🔧 Testing structured chat response...")
    
    test_message = "Explain what machine learning is in simple terms"
    print(f"👤 User: {test_message}")
    
    response = await structured_chat(test_message)
    
    if isinstance(response, ChatResponse):
        print(f"🤖 Assistant: {response.message}")
        print(f"📊 Confidence: {response.confidence}")
        print(f"🏷️  Keywords: {', '.join(response.keywords)}")
    else:
        print(f"❌ {response}")

# Run structured test
try:
    if 'structured_agent' in locals():
        await test_structured_chat()
    else:
        print("⚠️  Structured agent not initialized. Please run the previous cell first.")
except Exception as e:
    print(f"❌ Error during structured chat test: {e}")

## Key Features Demonstrated

1. **Basic Chat**: Simple question-answer interaction
2. **Structured Responses**: Using Pydantic models for type-safe outputs
3. **Error Handling**: Robust error handling for network and model issues
4. **Configuration Integration**: Uses shared config.py for consistent settings

## PydanticAI vs Other Frameworks

| Feature | PydanticAI | LangChain | Semantic Kernel |
|---------|------------|-----------|----------------|
| **Type Safety** | ✅ Native Pydantic models | ⚠️ Manual typing | ⚠️ Limited typing |
| **Structured Output** | ✅ Built-in | 🔧 Requires setup | 🔧 Custom implementation |
| **Async Support** | ✅ Native async/await | ✅ Supported | ✅ Supported |
| **Dependency Injection** | ✅ Built-in | ❌ Manual | ❌ Manual |
| **Validation** | ✅ Automatic | 🔧 Manual | 🔧 Manual |

## Next Steps

- Explore agent dependencies and dependency injection
- Implement multi-agent conversations
- Add function calling and tool integration
- Create more complex structured response models

In [ ]:
# Cleanup and summary
print("📋 Summary:")
print("✅ PydanticAI chat notebook completed")
print("🔧 Features demonstrated:")
print("   - Basic chat interaction")
print("   - Structured responses with Pydantic models")
print("   - Error handling and connection validation")
print("   - Integration with shared configuration")

print("\n🚀 To extend this notebook:")
print("   - Add function calling capabilities")
print("   - Implement conversation memory")
print("   - Create multi-agent workflows")
print("   - Add streaming responses")